In [373]:
import dash_leaflet as dl
import dash_leaflet.express as dlx
from dash import Dash, html, Output, Input,callback
import json
import geopandas as gpd
import plotly.express as px
import dash
from shapely.geometry import Point
from dash_extensions.javascript import assign
from dash import dcc
from shapely.geometry import shape, Point
import pandas as pd
import plotly.graph_objs as go
import dash_bootstrap_components as dbc
with open('Indian_states.geojson', 'r') as file:
    dev_borders = json.load(file)

In [374]:
df = pd.read_csv('EnrollByMEd.csv')
df1 = pd.read_csv('enrollment.csv')
df1 = df1.sort_values(by="ENRBTOT", ascending=True).head(10)
df2 = pd.read_csv('Facilities.csv')
# print(df)
# Define unique school types
school_types = df['School Type'].unique()
#********************************
state_name='Gujarat'
df3 = pd.read_csv("stream.csv")
df3=df3[df3['STATE']==state_name]
# print(df3)
df4 = pd.read_csv('StudentToTeacher.csv')
# Filter data for specific state and year
year = '2016-17'
df4 = df4[df4['STATENAME'] == state_name]
df5 = pd.read_csv('TeachersByQual.csv')
data1=df5[df5['State']==state_name]
# print(data1)
#********************************

In [375]:
school_types1 = ['1', '2', '3', '4', '5']    #6,7
get_name={
            '1': 'Primary',
            '2': 'Primary & Upper Primary',
            '3': 'Primary, Upper Primary & SEC/HSEC',
            '4': 'Upper Primary',
            '5': 'Upper Primary & SEC/HSEC'#,
            # '6': 'Primary, Upper Primary & SEC',
            # '7': 'Upper Primary & SEC'
        }
# Calculate maximum value across all school types for each qualification
max_values = []
for school_type in school_types1:
    qualifications = [f'TCHHS{school_type}', f'TCHGD{school_type}', f'TCHPG{school_type}', f'TCHMD{school_type}']
    max_values.append(data1[qualifications].values.max())

In [376]:
def create_bar_graph(title, x_data_boys, x_data_girls, y_data, color_boys, color_girls, show_yaxis_label=False):
    return dcc.Graph(
        figure={
            'data': [
                go.Bar(
                    x=x_data_boys,
                    y=y_data,
                    orientation='h',
                    marker=dict(color=color_boys),  # Set color of boys bars
                    name='Boys'
                ),
                go.Bar(
                    x=x_data_girls,
                    y=y_data,
                    orientation='h',
                    marker=dict(color=color_girls),  # Set color of girls bars
                    name='Girls'
                )
            ],
            'layout': go.Layout(
                title=title,
                xaxis=dict(title='Count', showgrid=False),  # Hide gridlines
                yaxis=dict(title='Stream' if show_yaxis_label else '', showgrid=False, showticklabels=show_yaxis_label),  # Hide gridlines and tick labels
                margin=dict(l=100, r=100, t=50, b=50),  # Adjust margins as needed
                barmode='stack'  # Stack bars on top of each other
            )
        }
    )


In [377]:
# Select specific columns
selected_columns = ['Primary Only_R', 'Primary with Upper Primary_R', 'Primary with upper Primary Sec/H.Sec_R', 
                    'Upper Primary Only_R', 'Upper Primary with Sec./H.Sec_R', 'Primary with upper Primary Sec_R', 
                    'Upper Primary with Sec._R']
# Calculate sum of values for each column
# Define custom colors for each bar
# custom_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']
custom_colors = [
    '#CC0033',
    '#E40039',
    '#F2003F',
    '#FF0044',
    '#FF3355',
    '#FF6677',
    '#FF99AA'
]

In [378]:
heading_style = {'color': 'white', 'text-align': 'center', 'font-size': '5rem'}
button_style = {'display': 'inline-block', 'width': '24px', 'height': '24px', 'margin-left': '5px'}
graph_style = {'width': '48%', 'display': 'inline-block', 'vertical-align': 'top', 'margin-right': '2%'}
graph_container_style = {'padding': '20px', 'text-align': 'center'}
box_container_style = {'text-align': 'right'}
# Define layout
def create_Page2_layout():
    dff = df4[(df4['YEAR'] == year)]
    selected_df = dff[selected_columns]
    sum_values = selected_df.sum()
    global state_name
    return html.Div([
        
        # First section
        html.Div([
            html.H1("{}".format(state_name), style=heading_style),
            html.Div(style=box_container_style, children=[
                html.A(html.Div(style={**button_style, 'background-color': 'red'}),href='/'),
                html.Div(style={**button_style, 'background-color': 'blue'}),
                html.Div(style={**button_style, 'background-color': 'green'}),
            ]),
        ], style={'background-color': 'black', 'padding': '20px'}),
        
        # Second section
        html.Div([
            # First row of graphs
            html.Div([
                html.Div([

                    html.Div([
                        html.H1("STREAM SELECTION PREFERENCE BY STUDENTS"),
                        dcc.RadioItems(
                            id='class-selector',
                            options=[
                                {'label': '11th Class', 'value': '11'},
                                {'label': '12th Class', 'value': '12'}
                            ],
                            value='11',  # Default value
                            labelStyle={'display': 'inline-block'}
                        ),
                        html.Div(id='graphs-container')
                    ])

                ]),     
            ], className='row', style=graph_container_style),
            # Second row of graphs
            html.Div([
                html.Div([
                    html.Div([
                        html.Div([
                            dcc.Graph(
                                id='bar-chart',
                                figure={
                                    'data': [
                                        go.Bar(
                                            x=sum_values.index,
                                            y=sum_values.values,
                                            marker_color=custom_colors  # Set color for each bar individually
                                        )
                                    ],
                                    'layout': go.Layout(
                                        title='Distribution of Student to Teacher Ratio by Category',
                                        xaxis={'title': 'Student to Teacher Ratio'},
                                        yaxis={'title': 'Type of school'},
                                        plot_bgcolor='#f5f5f5',  # Set plot background color
                                        barmode='group'  # Set mode for stacking bars
                                    )
                                }
                            ),
                        ], style={'display': 'inline-block', 'width': '50%'}),  # Place bar chart in a div with 50% width

                        html.Div([
                            dcc.Graph(id='line-chart')
                        ], style={'display': 'inline-block', 'width': '50%'})  # Place line chart in a div with 50% width
                        ])
                ]),
                
            ], className='row', style=graph_container_style),
            
            # Third of graphs
            html.Div([
                html.Div([
    dcc.Dropdown(
        id='school-type-dropdown_1',
        options=[
            {'label': 'Primary', 'value': '1'},
            {'label': 'Primary & Upper Primary', 'value': '2'},
            {'label': 'Primary, Upper Primary & SEC/HSEC', 'value': '3'},
            {'label': 'Upper Primary', 'value': '4'},
            {'label': 'Upper Primary & SEC/HSEC', 'value': '5'}#,
            # {'label': 'Primary, Upper Primary & SEC', 'value': '6'},
            # {'label': 'Upper Primary & SEC', 'value': '7'}
        ],
        value='1',
        style={'width': '80%'} 
          # Default value
    ),
    # dcc.Graph(id='teacher-qualification-graph')
    html.Div(
        dcc.Graph(id='teacher-qualification-graph'),
        style={'width': '80%'}  # Adjust the width here
    )
]),
                
            ], className='row', style=graph_container_style),
        ]),
    ], style={'padding': '20px'})
# lay = create_Page2_layout()

In [379]:
graph =   html.Div([

html.Div([
    html.Div([
        
        dcc.Dropdown(
            id='school-type-dropdown',
            options=[{'label': st, 'value': st} for st in school_types],
            value=school_types[0],  # Default value
            clearable=False,
            style={'width': '50%', 'margin': 'auto', 'textAlign': 'center'}
        ),
        dcc.RadioItems(
            id='medium-radio',
            options=[
                {'label': 'Hindi', 'value': 'Hindi'},
                {'label': 'English', 'value': 'English'}
            ],
            value='Hindi',  # Default value
            labelStyle={'display': 'inline-block','color':'white','fontSize':'20px'},
            style={'textAlign': 'center','width':'50%','margin':'auto'}
        ),
    ], style={'textAlign': 'center','display':'flex'}),
    
    html.Div([
        dcc.Graph(
            id='school-count-graph',
            config={'displayModeBar': False},
            style={'height': '60vh', 'width': '40vw'}
        ),
        dcc.Graph(
            id='pie-chart',
            config={'displayModeBar': False},
            style={'height': '60vh', 'width': '40vw'}
        )
    ], style={'display': 'flex', 'justifyContent': 'center', "background":"white"})
], style={"width":"90vw", "margin-top":"130px"})


], style={"display":"flex", "justify-content":"center"})

In [380]:
# Load CSV file into a pandas DataFrame
# Define colors for bars
boys_color = '#e5c185'  # Royal Blue
girls_color = 'rgba(255, 105, 180, 0.7)'  # Pink
graph1= html.Div([
    dcc.Graph(id='enrollment-pyramid',style={})
],style={'left':'2%','height':'50%','width':'40%','flex':'50%','margin':'5%'})


In [381]:
# Read data from CSV
# Define custom color scale
# colors = ['rgb(26, 118, 255)', 'rgb(55, 83, 109)', 'rgb(190, 22, 144)', 'rgb(100, 200, 50)', 'rgb(255, 0, 0)', 
#           'rgb(0, 255, 0)', 'rgb(0, 0, 255)', 'rgb(255, 255, 0)', 'rgb(255, 0, 255)', 'rgb(0, 255, 255)', 
#           'rgb(128, 0, 128)', 'rgb(128, 128, 0)', 'rgb(0, 128, 128)', 'rgb(128, 0, 0)', 'rgb(0, 128, 0)']

colors = [
    'rgb(0, 0, 139)',   # Dark Blue
    'rgb(16, 78, 139)', # Dark Slate Blue
    'rgb(0, 0, 205)',   # Medium Blue
    'rgb(65, 105, 225)',# Royal Blues
    'rgb(0, 191, 255)', # Deep Sky Blue
    'rgb(0, 206, 209)', # Dark Cyan
    'rgb(0, 255, 255)', # Cyan
    'rgb(255, 140, 0)', # Dark Orange
    'rgb(255, 165, 0)', # Orange
    'rgb(255, 218, 185)'# Peach
]


# Define layout of the app
graph2 = html.Div([
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': year, 'value': year} for year in df2['YEAR'].unique()],
        value=df2['YEAR'].min(),  # Set default value to the minimum year
        clearable=False,
    ),
    dcc.RadioItems(
        id='facility-radio',
        options=[
            {'label': 'Total Electricity', 'value': 'Total_Elec'},
            {'label': 'Total Computers', 'value': 'Total_Comp'},
            {'label': 'Total Play Grounds', 'value': 'Total_PG'},
            {'label': 'Total Schools', 'value': 'Total_Schools'}
        ],
        value='Total_Elec',  # Set default value to 'Total Electricity'
        labelStyle={'display': 'inline-block','color':'white','fontSize':'15px'},
        inline=True
    ),
    dcc.Graph(id='radial-bar-chart')
],style={'right':'2%','height':'35%','width':'40%','flex':'50%','margin':'5%'})

In [382]:
flexed_graphs=html.Div([graph1,graph2],style={'display':'flex', })

In [383]:
def point_in_polygon(point, polygon):
    """Check if a point is within a polygon"""
    from shapely.geometry import shape, Point
    polygon_shape = shape(polygon)
    # point = Point(point['coordinates'])
    return polygon_shape.contains(point)

def is_point_in(lat, lon, geojson=dev_borders):
    if not isinstance(geojson, dict) or 'features' not in geojson:
        raise ValueError("GeoJSON must be a dictionary with a 'features' key.")

    # Correct the order of lat and lon when creating the Point
    point = Point(lon, lat)  # Use (longitude, latitude)
    state_name=""
    # Create a GeoDataFrame from the GeoJSON features
    gdf = gpd.GeoDataFrame.from_features(geojson['features'])
    # Check if the point is within any of the geometries in the GeoDataFrame
    is_within = gdf.contains(point).any()
    if is_within:
        for feature in dev_borders['features']:
            polygon = feature['geometry']
            if point_in_polygon(point, polygon):
                state_name = feature['properties']['st_nm']
                # print(f"You clicked on {state_name}")
        # for i in range(36):
        #     gdf=gpd.GeoDataFrame(dev_borders['features'][i])
        #     if gdf.contains(point):
        #         state=print(dev_borders['features'][i]['properties']['st_nm'])
        #         break     
    return is_within,state_name


I will have to create a callback function which will change hover data in the map

In [384]:
classes = [60,65,70,75,80,85,90,95]
# colorscale = ['#6A040F','#9D0208','#D00000', '#FAA307','#FFBA08', '#D4D700', '#80B918', '#55A630']
colorscale = ['#caf0f8','#90e0ef','#48cae4', '#00b4d8','#0096c7', '#0077b6', '#023e8a', '#03045e']

style = dict(weight=2, opacity=1, color='white', dashArray='3', fillOpacity=0.7)
# Create colorbar.
ctg = ["{}+".format(cls, classes[i + 1]) for i, cls in enumerate(classes[:-1])] + ["{}+".format(classes[-1])]
colorbar = dlx.categorical_colorbar(categories=ctg, colorscale=colorscale, width=300, height=20, position="bottomleft",style={'color':'white'})
# Geojson rendering logic, must be JavaScript as it is executed in clientside.
style_handle = assign("""function(feature, context){
    const {classes, colorscale, style, colorProp} = context.hideout;  // get props from hideout
    const value = feature.properties[colorProp];  // get value the determines the color
    for (let i = 0; i < classes.length; ++i) {
        if (value > classes[i]) {
            style.fillColor = colorscale[i];  // set the fill color according to the class
        }
    }
    return style;
}""")

In [385]:
name="Literacy rate"
def get_info(feature=None):
    header = [html.H4("{}".format(name))]
    if not feature:
        return header + [html.P("Hover over a state")]
    return header + [html.B(feature["properties"]["st_nm"]), html.Br(),
                     "{} -  {:.3f} ".format(name,feature["properties"]["density"])] + [html.Div(id="result")]
def get_category(value):
    # global name
    header = [html.H4("{}".format(value))]
    return header
def get_state(feature=None):
    if not feature:
        return "Hover Over a State"
    return feature["properties"]["st_nm"]
def get_value(feature=None):
    if not feature:
        return "Hover Over a State"
    return feature["properties"]["density"]
info = html.Div(children=get_info(), id="info", className="info",
                style={"position": "absolute", "top": "10%", "left": "30px", "zIndex": "1000","color":"white"})
card = html.Div(className="hover-state",children=[dbc.Card(id="state_card",children=get_state(),className="mb-3")],style={'position':'absolute','top':'35%','right':'8%','zIndex':'1000','fillOpacity':'0.7','width':'40%','height':'10%','text-align': 'center'})
card1 = html.Div(className="hover-state",children=[dbc.Card(id="Category_Card",children=get_category("Literacy Rate"),className="mb-3")],style={'margin':'10px',"height": "50%", "width": "55%", "font-weight":"900"})
card2 = html.Div(className="hover-state",children=[dbc.Card(id="Value_card",children=get_value(),className="mb-3")],style={'margin':'10px',"height": "50%", "width": "55%", "font-weight":"900"})
Card=html.Div([card1,card2],style={'display':'flex','position':'absolute','top':'40%','right':'3%','zIndex':'1000','color':'blue','fillOpacity':'0.7','width':'40%','height':'50%','text-align': 'center','margin':'5%','justify-content': 'center'})
# card = html.Div(id="horizontal_card",children=[html.H1("State Name")],style={'position':'absolute','top':'5%','right':'5%','zIndex':'1000','color':'blue','background-color':'white','fillOpacity':'0.7','width':'20%'})
heading = html.Div([html.H1("INDIAN EDUCATION ANALYSIS"),html.Button('Compare', id='State_Comp')],style={'color':'white','position':'relative','left':'2%','font-size':'45px', "font-weight":"900",'text-align': 'center'})
# heading = html.Div([html.H1("Indian Education Analysis"),html.Button('Compare', id='State_Comp')],style={'display':'flex','color':'white','position':'relative','left':'40%','font-size':'20px','text-align': 'center'})
map_context = [
             dl.LayerGroup(id="layer"),
            dl.GeoJSON(
                data=dev_borders,  # Use the loaded GeoJSON data
                id="dev_layer",
                style=style_handle,
                hideout=dict(colorscale=colorscale, classes=classes, style=style, colorProp="density"),
                # Default style for the paths
                hoverStyle={"weight": 5, "color": "orange", "dashArray": ""},  # Style for hovered countries
            ),heading,colorbar,card,Card,
            # dl.LocateControl(
            #     locateOptions={'enableHighAccuracy': True, 'drawMarker': True, 'flyTo': True, 'showCompass': True}),
            # dl.EasyButton(icon="fa-globe", title="Map Switch", id="site_management_satellite_button"),
            dcc.RadioItems(id="radio_mirchi",options=[{'label':'Female Literacy Rate','value':'Female Literacy Rate'},{'label':'Male Literacy Rate','value':'Male Literacy Rate'},{'label':'Ratio','value':'Ratio'},{'label': 'Literacy Rate','value':'Literacy Rate'},{'label':'Transition Rate','value':'Transition Rate'}],className="radio-button" ,value='Literacy Rate',style={'position':'absolute','bottom':'10%','left': 0,'background-color':'#f0f0f0'})
        ]

new_center=[28,101]

In [386]:
app = dash.Dash(__name__)

map_click function only needs to redirect to another web-page and not put the markers like it is doing now

In [387]:
literacy_rates = {"Andaman & Nicobar Island": 86.63,"Arunanchal Pradesh": 66.95,"Assam": 72.19,"Bihar": 61.80,"Chandigarh": 86.43,"Chhattisgarh": 71.04,"Dadara & Nagar Havelli": 77.65,"Daman & Diu":77.65,"Goa": 87.40,"Gujarat": 78.03,"Haryana": 76.64,
    "Himachal Pradesh": 83.78,"Jammu & Kashmir":67.16,"Jharkhand": 67.63,"Karnataka": 75.60,"Kerala": 94.00,"Lakshadweep": 91.85,
    "Madhya Pradesh": 70.63,"Maharashtra": 82.34,"Manipur": 79.85,"Meghalaya": 75.48,"Mizoram": 91.58,"Nagaland": 79.55,"Delhi": 86.21,
    "Puducherry": 86.55,"Punjab": 76.68,"Rajasthan": 66.11,"Sikkim": 81.42,"Tamil Nadu": 80.33,"Telangana": 66.50,"Tripura": 87.22,
    "Uttar Pradesh": 69.72,"Uttarakhand": 79.63,"West Bengal": 77.08,"Odisha": 72.87,"Andhra Pradesh": 67.0
}

Male_Literacy_Rate = {
    "Andaman & Nicobar Island": 90.11, "Andhra Pradesh": 75.56, "Arunanchal Pradesh": 73.69, "Assam": 78.81,
    "Bihar": 73.39, "Chandigarh": 90.54, "Chhattisgarh": 81.45, "Dadara & Nagar Havelli": 86.46,
    "Daman & Diu": 91.48, "Delhi": 91.03, "Goa": 92.81, "Gujarat": 87.23, "Haryana": 85.38, "Himachal Pradesh": 90.83,
    "Jammu & Kashmir": 78.26, "Jharkhand": 78.45, "Karnataka": 82.85, "Kerala": 96.02, "Lakshadweep": 96.11,
    "Madhya Pradesh": 80.53, "Maharashtra": 89.82, "Manipur": 86.49, "Meghalaya": 77.17, "Mizoram": 93.72,
    "Nagaland": 83.29, "Odisha": 82.4, "Puducherry": 92.12, "Punjab": 81.48, "Rajasthan": 80.51, "Sikkim": 87.29,
    "Tamil Nadu": 86.81, "Telangana": 74.95, "Tripura": 92.18, "Uttar Pradesh": 79.24, "Uttarakhand": 88.33,
    "West Bengal": 82.67
}

Female_Literacy_Rate = {
    "Andaman & Nicobar Island": 81.84, "Andhra Pradesh": 59.74, "Arunanchal Pradesh": 59.57, "Assam": 67.27,
    "Bihar": 53.33, "Chandigarh": 81.38, "Chhattisgarh": 60.59, "Dadara & Nagar Havelli": 65.93,
    "Daman & Diu": 79.59, "Delhi": 80.93, "Goa": 81.84, "Gujarat": 70.73, "Haryana": 66.77, "Himachal Pradesh": 76.6,
    "Jammu & Kashmir": 58.01, "Jharkhand": 56.21, "Karnataka": 68.13, "Kerala": 91.98, "Lakshadweep": 88.25,
    "Madhya Pradesh": 60.02, "Maharashtra": 75.48, "Manipur": 73.17, "Meghalaya": 73.78, "Mizoram": 89.4,
    "Nagaland": 76.69, "Odisha": 64.36, "Puducherry": 81.22, "Punjab": 71.34, "Rajasthan": 52.66, "Sikkim": 76.43,
    "Tamil Nadu": 73.86, "Telangana": 57.92, "Tripura": 83.15, "Uttar Pradesh": 59.26, "Uttarakhand": 70.7,
    "West Bengal": 71.16
}

Ratio = {'Andaman & Nicobar Island': 60.0, 'Andhra Pradesh': 73.64787300094889, 'Arunanchal Pradesh': 64.66927250399245, 
'Assam': 74.21009900438189, 'Bihar': 100.0, 'Chandigarh': 64.71537813549867, 'Chhattisgarh': 74.92901797698791, 
'Dadara & Nagar Havelli': 95.37952083846164, 'Daman & Diu': 74.86401827532343, 'Delhi': 70.55612034642884, 'Goa': 61.15998140901059, 
'Gujarat': 75.67769710679082, 'Haryana': 72.93996146328978, 'Himachal Pradesh': 65.27190462227168, 'Jammu & Kashmir': 66.18278472140587, 
'Jharkhand': 88.78162981776707, 'Karnataka': 76.12303924109813, 'Kerala': 69.06575876813936, 'Lakshadweep': 60.116880803967305, 
'Madhya Pradesh': 81.45485579341371, 'Maharashtra': 69.72360015109574, 'Manipur': 70.99963413815907, 'Meghalaya': 71.1422740066328, 
'Mizoram': 64.3376922378096, 'Nagaland': 71.91267884888266, 'Odisha': 79.62981189992598, 'Puducherry': 74.59037438160387, 
'Punjab': 77.98971681690328, 'Rajasthan': 78.02882135737099, 'Sikkim': 61.52741651158464, 'Tamil Nadu': 72.06762035851267, 
'Telangana': 77.98177277196672, 'Tripura': 69.63008891742487, 'Uttar Pradesh': 83.64400335563406, 'Uttarakhand': 68.13894743113346,
 'West Bengal': 81.16809965830025}
Transition_rates = {
    "Andaman & Nicobar Island": 98.36, "Andhra Pradesh": 95.92, "Arunanchal Pradesh": 98.04, "Assam": 93.15,
    "Bihar": 86.2, "Chandigarh": 98.6, "Chhattisgarh": 93.07, "Dadara & Nagar Havelli": 98.18, "Daman & Diu": 99.1,
    "Delhi": 99.5, "Goa": 97.8, "Gujarat": 97.81, "Haryana": 97.02, "Himachal Pradesh": 97.85, "Jammu & Kashmir": 93.25,
    "Jharkhand": 80.19, "Karnataka": 94.26, "Kerala": 0, "Lakshadweep": 97.01, "Madhya Pradesh": 87.43,
    "Maharashtra": 98.95, "Manipur": 87.7, "Meghalaya": 0, "Mizoram": 84.13, "Nagaland": 88, "Odisha": 88.76,
    "Puducherry": 0, "Punjab": 97.61, "Rajasthan": 88.67, "Sikkim": 94.89, "Tamil Nadu": 95.39, "Telangana": 95.92,
    "Tripura": 92.1, "Uttar Pradesh": 76.92, "Uttarakhand": 95.85, "West Bengal": 92.42
}
# print(len(literacy_rates))

In [388]:
# maping={'Madhya Pradesh' : 'MADHYA PRADESH'}

In [389]:
app.layout = html.Div(id="Main_Div",children=[html.Div(children=[dl.Map(center=new_center,scrollWheelZoom=False,maxZoom=5,zoom=5,zoomSnap=0.25,zoomControl=False,children=map_context,id='map',style={'width': '100%', 'height': '100vh','background-color':'rgba(255,255,255,0)'}),graph,flexed_graphs],style={'background':"#23272E"})])
@app.callback(Output("Main_Div", "children"), [Input("map", "clickData"),Input("State_Comp","n_clicks")], prevent_initial_call=True)
def map_click(click_lat_lng,value):
    if value is not None:
        print("V")
        return html.H1("Page3")
    elif click_lat_lng is not None:
        print("M")
        coordinates = click_lat_lng['latlng']
        lat, lon = coordinates.values()
        point_in,st_nm=is_point_in(lat,lon,dev_borders)
        if point_in:
            global state_name 
            global df3
            global df4
            global df5
            global data1
            global max_values
            global school_types1
            state_name = st_nm
            df3 = pd.read_csv("stream.csv")
            df3=df3[df3['STATE']==state_name]
            # print(df3)
            df4 = pd.read_csv('StudentToTeacher.csv')
            # Filter data for specific state and year
            year = '2016-17'
            df4 = df4[df4['STATENAME'] == state_name]
            df5 = pd.read_csv('TeachersByQual.csv')
            data1=df5[df5['State']==state_name]
            for school_type in school_types1:
                qualifications = [f'TCHHS{school_type}', f'TCHGD{school_type}', f'TCHPG{school_type}', f'TCHMD{school_type}']
                max_values.append(data1[qualifications].values.max())
            return create_Page2_layout()
            # html.H1(state_name)
            # print("IGI I'm Going In")
            # Return a list of new elements to add to the 'layer' component
            # return dl.Marker(
            #         position=[lat, lon],
            #         children=dl.Tooltip("Chosen 🗺 Location: ({:.5f}, {:.5f})".format(lat, lon)),
            #         icon={'icon':'default',
            #               'iconRetinaUrl': 'assets/site_icon.png',
            #               'iconSize': [40, 40], 'iconAnchor': [12, 12], 'popupAnchor': [0, -12],
            #               'shadowUrl': None, 'shadowSize': None, 'shadowAnchor': None}
            #     )
        return dash.no_update
    else:print("click_lat_lang is None")

# @callback(Output("dev-layer","hoverStyle"),[Input("map","clickData")],prevent_intial_call=True)
# def map_hover(click_lat_lang):
#     if click_lat_lang is not None:
#         coordinates = click_lat_lang['latlng']
#         lat, lon = coordinates.values()
#         point_in,name=is_point_in(lat,lon,dev_borders)
#         if point_in:
#             return {"weight": 5, "color": "orange", "dashArray":""} #make the update dynamic to display state's name    
@app.callback(Output("Value_card","children"),Output("state_card","children"),Input("dev_layer", "hoverData"))
def info_hover(feature):
    return get_value(feature),get_state(feature)

# @callback(Output("result","children"),Input("radio_mirchi","value"))
# def radio_mirchi(value):
#     return value

@app.callback(Output("Category_Card","children"),Output("dev_layer","data"),
          Input("radio_mirchi","value"))
def radio_mirchi(value):
    print("radio_mirchi")
    global name
    if value == 'Literacy Rate':
        for i in range(36):
            dev_borders['features'][i]['properties']['density']=literacy_rates[dev_borders['features'][i]['properties']['st_nm']]
        name="Literacy Rate" #change the box to say Literacy rate
        # get_info()
    elif value == 'Transition Rate':
        for i in range(36):
            dev_borders['features'][i]['properties']['density']=Transition_rates[dev_borders['features'][i]['properties']['st_nm']]
        name="Transition Rate" #Change the box to say Transition Rate
        # info_hover()
    elif value == 'Ratio':
        for i in range(36):
            dev_borders['features'][i]['properties']['density']=Ratio[dev_borders['features'][i]['properties']['st_nm']]
        name="Ratio" #Change the box to say Transition Rate
    elif value == 'Male Literacy Rate':
        for i in range(36):
            dev_borders['features'][i]['properties']['density']=Male_Literacy_Rate[dev_borders['features'][i]['properties']['st_nm']]
        name="Male Literacy Rate" #Change the box to say Transition Rate
    elif value == 'Female Literacy Rate':
        for i in range(36):
            dev_borders['features'][i]['properties']['density']=Female_Literacy_Rate[dev_borders['features'][i]['properties']['st_nm']]
        name="Female Literacy Rate" #Change the box to say Transition Rate
    return get_category(value),dev_borders 
@app.callback(
    [Output('school-count-graph', 'figure'),
     Output('pie-chart', 'figure')],
    [Input('school-type-dropdown', 'value'),
     Input('medium-radio', 'value'),
     Input('school-count-graph', 'clickData')]
)
def update_graph(selected_school_type, selected_medium, click_data):
    # Filter DataFrame based on selected school type
    filtered_df = df[df['School Type'] == selected_school_type]
    
    # Group by year and calculate sum of selected medium
    medium_df = filtered_df.groupby('Year')[selected_medium].sum().reset_index()

    # Create bar graph
    bar_fig = {
        'data': [{
            'x': medium_df['Year'],
            'y': medium_df[selected_medium],
            'type': 'bar',
            'marker': {'color': '#cc0033'}  # Set bar color
        }],
        'layout': {
            'title': f'Number of {selected_medium} Medium Schools for {selected_school_type} Over Years',
            'xaxis': {'title': 'Year'},
            'yaxis': {'title': f'Number of {selected_medium} Medium Schools'},
            'plot_bgcolor': 'rgba(240, 240, 240, 1)',  # Set plot background color
            'paper_bgcolor': 'rgba(240, 240, 240, 1)',  # Set paper background color
            'bargap': 0.2,  # Set gap between bars
            'bargroupgap': 0.1,  # Set gap between groups of bars
            'barmode': 'overlay',  # Overlay bars to control width individually
            'width': 0.4  # Set width of each bar
        }
    }

    color_sequence = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    # Create pie chart for selected year
    if click_data is not None:
        # print(click_data)
        selected_year = click_data['points'][0]['x']
    else:
        selected_year='2012-13'
    pie_df=df[df['Year']==selected_year]
    # print(pie_df)
    pie_counts = pie_df[selected_medium].tolist()
    pie_labels = pie_df['School Type'].tolist()
    pie_fig = go.Figure(data=[go.Pie(labels=pie_labels, values=pie_counts)])
    pie_fig.update_layout(title=f'School Type Distribution for {selected_medium} Medium in {selected_year}',plot_bgcolor='rgba(240, 240, 240, 0.7)',paper_bgcolor='rgba(240, 240, 240, 0.7)')
    pie_fig.update_traces(hole=0.6, marker=dict(colors=color_sequence))

    return bar_fig, pie_fig


# Define callback to update the graph
@app.callback(
    Output('enrollment-pyramid', 'figure'),
    [Input('enrollment-pyramid', 'id')]
)
def update_pyramid(_):
    # Create the pyramid plot
    data = [
        go.Bar(
            y=df1['State'],
            x=df1['ENRBTOT'],
            orientation='h',
            name='Boys Enrollment',
            marker=dict(color=boys_color),
            textposition='auto',
            width=0.8  # Increase width of bars
        ),
        go.Bar(
            y=df1['State'],
            x=-df1['ENRGTOT'],
            orientation='h',
            name='Girls Enrollment',
            marker=dict(color=girls_color),
            textposition='auto',
            width=0.8  # Increase width of bars
        )
    ]

    layout = go.Layout(
        barmode='relative',
        title='Enrollment by Gender and State',
        xaxis=dict(title='Enrollment', range=[-2 * max(df1['ENRGTOT']), 2 * max(df1['ENRBTOT'])]),
        yaxis=dict(title='State', autorange='reversed'),
        legend=dict(orientation='h', x=0.5, y=-0.1),
        plot_bgcolor='rgba(240, 240, 240, 0.7)',  # Set plot background color
        paper_bgcolor='rgba(240, 240, 240, 0.7)'  # Set paper background color
    )

    return {'data': data, 'layout': layout}


# Define callback to update radial bar chart
@app.callback(
    Output('radial-bar-chart', 'figure'),
    [Input('year-dropdown', 'value'),
     Input('facility-radio', 'value')]
)
def update_radial_chart(selected_year, selected_facility):
    # Filter data based on selected year and facility
    filtered_df = df2[(df2['YEAR'] == selected_year)]
    
    # Get top 15 states by selected facility
    top_15_states = filtered_df.groupby('STATNAME')[selected_facility].sum().nlargest(10)
    
    # Create radial bar chart
    fig = go.Figure()
    for state, value in top_15_states.items():
        # Modify color to make it slightly darker
        color = colors[top_15_states.index.get_loc(state) % len(colors)]
        color_dark = "rgb" + str(tuple([min(max(int(c * 0.8), 0), 255) for c in eval(color[3:])]))
        
        fig.add_trace(go.Barpolar(
            r=[value],
            theta=[state],
            marker_color=color_dark,  # Apply slightly darker color
            opacity=0.7,
            name='',
            hoverinfo='text',  # Show state name and value on hover
            hovertext=f'{state}: {value}',
            text=[state],
        ))
    
    fig.update_layout(
        title=f'Top 15 States by {selected_facility} in {selected_year}',
        font_size=16,
        polar=dict(
            bgcolor='white',
            radialaxis=dict(
                visible=True,
                showline=False,
                showticklabels=False,
                range=[0, top_15_states.max()]  # Adjust range as needed
            )),
        showlegend=False,  # Remove legend
        plot_bgcolor='rgba(240, 240, 240, 0.7)',  # Set plot background color
        paper_bgcolor='rgba(240, 240, 240, 0.7)'  # Set paper background color
    )
    
    return fig


In [390]:
@app.callback(
    Output('graphs-container', 'children'),
    [Input('class-selector', 'value')]
)
def update_graphs(selected_class):
    graphs = []
    if(selected_class=='11'):
        graphs.append(html.Div(create_bar_graph('Arts Stream', df3['arts_c11_b'] , df3['arts_c11_g'] , df3['YEAR'], '#8B0000', '#FF6347', show_yaxis_label=True), style={'width': '33%', 'display': 'inline-block'})),
        graphs.append(html.Div(create_bar_graph('Commerce Stream', df3['com_c11_b'] , df3['com_c11_g'] , df3['YEAR'], '#0000FF', '#87CEEB',), style={'width': '33%', 'display': 'inline-block'})),
        graphs.append(html.Div(create_bar_graph('Science Stream', df3['sci_c11_b'] , df3['sci_c11_g'] , df3['YEAR'], '#008000', '#90EE90'), style={'width': '33%', 'display': 'inline-block'}))
    if(selected_class=='12'):
        graphs.append(html.Div(create_bar_graph('Arts Stream', df3['arts_c12_b'] , df3['arts_c12_g'] , df3['YEAR'], '#8B0000', '#FF6347', show_yaxis_label=True), style={'width': '33%', 'display': 'inline-block'})),
        graphs.append(html.Div(create_bar_graph('Commerce Stream', df3['com_c12_b'] , df3['com_c12_g'] , df3['YEAR'], '#0000FF', '#87CEEB',), style={'width': '33%', 'display': 'inline-block'})),
        graphs.append(html.Div(create_bar_graph('Science Stream', df3['sci_c12_b'] , df3['sci_c12_g'] , df3['YEAR'], '#008000', '#90EE90'), style={'width': '33%', 'display': 'inline-block'}))
    return graphs

# *********************************************** From Graph 1 end   **************************************************************


# *********************************************** From Graph 2 start   **************************************************************

# Define callback to update line chart based on bar chart click
@app.callback(
    Output('line-chart', 'figure'),
    [Input('bar-chart', 'clickData')]
)
def update_line_chart(clickData):
    if clickData is None:
        school_data_teachers = df4['Total_T']
        school_data_enrollments = df4['Total_E']
        school_type = 'All_Schools'
    else:
        school_type = clickData['points'][0]['x']
        school_type_teachers = school_type.replace('_R', '_T')
        school_type_enrollments = school_type.replace('_R', '_E')
        # Filter data for selected school type
        school_data_teachers = df4[school_type_teachers]
        school_data_enrollments = df4[school_type_enrollments]
    # Create line chart
    line_chart = go.Figure()
    # Add trace for teachers
    line_chart.add_trace(go.Scatter(x=df4['YEAR'], y=school_data_teachers, mode='lines+markers', name='Teachers',
                                     yaxis='y1'))  # Assigning to yaxis='y1' for the first y-axis
    # Add trace for enrollments
    line_chart.add_trace(go.Scatter(x=df4['YEAR'], y=school_data_enrollments, mode='lines+markers', name='Enrollments',
                                     yaxis='y2'))  # Assigning to yaxis='y2' for the second y-axis

    line_chart.update_layout(title=f'Number of Teachers and Enrollments Over the Years for {school_type}',
                             xaxis_title='Year',
                             yaxis=dict(title='Number of Teachers', side='left'),  # Define properties for y-axis 1
                             yaxis2=dict(title='Number of Enrollments', side='right', overlaying='y'))  # Define properties for y-axis 2

    return line_chart

In [391]:
qualification_colors = {
    'TCHHS': 'red',
    'TCHGD': 'orange',
    'TCHPG': 'green',
    'TCHMD': 'green'
}
legend_names = {
    'TCHHS': 'Higher Sec.',
    'TCHGD': 'Grad.',
    'TCHPG': 'PG',
    'TCHMD': 'MD'
}
# Define callback to update the graph based on dropdown selection
@app.callback(
    Output('teacher-qualification-graph', 'figure'),
    [Input('school-type-dropdown_1', 'value')]
)
def update_graph(selected_school_type):
    hs_qualifications = [f'TCHHS{selected_school_type}', f'TCHGD{selected_school_type}',f'TCHPG{selected_school_type}', f'TCHMD{selected_school_type}']
    # print(hs_qualifications)
    hs_qualifications = sorted(hs_qualifications, key=lambda q: data1[q].iloc[0])[::-1]
    traces = []

    for qualification in hs_qualifications:
        trace = go.Scatter(
            # x=data1['Year'],
            x=['2011','2012','2013','2014','2015','2016'],
            y=data1[qualification],
            mode='lines+markers',
            fill='tozeroy',  # Fill area below the line
            name=legend_names[qualification[:-1]],
            line=dict(color=qualification_colors[qualification[:-1]])
        )
        traces.append(trace)
    max_value = max(max_values)
    layout = go.Layout(
        title=f'Teachers Qualification for {get_name[selected_school_type]}',
        xaxis=dict(title='Year'),
        yaxis=dict(title='Number of Teachers',range=[0, max_value + 10])
    )

    return {'data': traces, 'layout': layout}

In [392]:
if __name__ == '__main__':
    app.run_server(debug=True,port=8052)

radio_mirchi
